In [ ]:
!pip install langchain openai pandas


In [ ]:
import pandas as pd
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 🚩 Configurar tu API key de OpenAI
import os
os.environ["OPENAI_API_KEY"] = "tu-api-key-aquí"

# 🚩 Cargar el CSV de términos y contextos
ruta_csv = '/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/terminos_contextos.csv'
df = pd.read_csv(ruta_csv)

# 🚩 Definir el modelo
llm = OpenAI(temperature=0.3, model_name='gpt-4')

# 🚩 Crear el prompt
prompt = PromptTemplate(
    input_variables=["termino", "contextos"],
    template="""
Eres un experto en lingüística jurídica.
Dado el conjunto de contextos de uso para el término "{termino}",
sintetiza una ventana de contexto de máximo 100 palabras que explique claramente el significado y uso del término,
con el objetivo de facilitar su enlazado con otros conceptos.

Contextos:
{contextos}

Respuesta:
"""
)

# 🚩 Crear la cadena de LangChain
chain = LLMChain(llm=llm, prompt=prompt)

# 🚩 Procesar cada término
resultados_finales = []

for idx, row in df.iterrows():
    termino = row['Termino']
    contextos = " ".join([str(row[col]) for col in df.columns if col.startswith('Contexto') and pd.notna(row[col])])

    if not contextos.strip():
        continue  # Saltar si no hay contextos

    respuesta = chain.run(termino=termino, contextos=contextos)
    resultados_finales.append({
        "Termino": termino,
        "Ventana_de_contexto": respuesta.strip()
    })

# 🚩 Guardar los resultados
df_resultado = pd.DataFrame(resultados_finales)
ruta_salida = '/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/terminos_ventanas_contexto.csv'
df_resultado.to_csv(ruta_salida, index=False, encoding='utf-8')

print(f'✅ Ventanas de contexto generadas y guardadas en {ruta_salida}')
